In [ ]:
!pip install vaderSentiment
!pip install -q transformers
!pip3 install emoji==0.6.0

In [ ]:
import csv
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from transformers import AutoTokenizer, AutoConfig

In [56]:
# Replace 'your_file.csv' with the actual file name
csv_file = '/content/reddit-labeled-output.csv'

sentences = []

with open(csv_file, 'r') as file:
    reader = csv.reader(file)

    for row in reader:
        # Assuming each row has at least 3 columns
        sentences.append(row[1]+". "+row[2]) if len(row[2]) > 0 else sentences.append(row[1])  # Indexing starts from 0

sentences = sentences[1:]

In [58]:
# https://github.com/cjhutto/vaderSentiment#python-demo-and-code-examples
sentiments = []
analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    sentiments.append(vs)

In [59]:
converted_sentiments = []
for sentiment in sentiments:
  val = sentiment['compound']
  if val >= 0.05:
    converted_sentiments.append(1)
  elif val < 0.05 and val > -0.05:
    converted_sentiments.append(0)
  else:
    converted_sentiments.append(-1)

In [ ]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
sentiment_task = pipeline("sentiment-analysis", model=f"cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer=AutoTokenizer.from_pretrained(f"cardiffnlp/twitter-roberta-base-sentiment-latest"))

sentiments_2 = []
epoch = 1
sequence_length = 128
for sentence in sentences:
  if epoch % 50 == 0:
    print(f"Epoch {epoch}")
  sentence_sentiment = []
  iterations = int(len(sentence)/sequence_length+0.5)
  for i in range(iterations-1):
    sentence_sentiment.append(sentiment_task(sentence[sequence_length*i:sequence_length*(i+1)]))
  sentence_sentiment.append(sentiment_task(sentence[sequence_length*(iterations-1):]))
  sentiments_2.append(sentence_sentiment)
  epoch += 1

In [45]:
def calculateSentiment(sentiment):
  sent_sum_pos = 0
  sent_sum_neu = 0
  sent_sum_neg = 0
  for i in range(len(sentiment)):
    if sentiment[i][0]['label'] == 'positive':
      sent_sum_pos += sentiment[i][0]['score']
    elif sentiment[i][0]['label'] == 'neutral':
      sent_sum_neu += sentiment[i][0]['score']
    else:
      sent_sum_neg += sentiment[i][0]['score']

  mag_score = max(sent_sum_pos,sent_sum_neu,sent_sum_neg)
  sent = 0

  if mag_score == sent_sum_pos:
    sent = 1
  elif mag_score == sent_sum_neu:
    sent = 0
  else:
    sent = -1

  return {'sent': sent,
          'score': mag_score/len(sentiment)
          }

sentiment_results_2 = []
for sentiment in sentiments_2:
  sentiment_results_2.append(calculateSentiment(sentiment))

converted_sentiments_2 = []
for result in sentiment_results_2:
  converted_sentiments_2.append(result['sent'])

In [ ]:
# https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

sentiments_3 = []
epoch = 1
sequence_length = 128
for sentence in sentences:
  if epoch % 50 == 0:
    print(f"Epoch {epoch}")
  sentence_sentiment = []
  iterations = int(len(sentence)/sequence_length+0.5)
  for i in range(iterations-1):
    sentence_sentiment.append(pipe(sentence[sequence_length*i:sequence_length*(i+1)]))
  sentence_sentiment.append(pipe(sentence[sequence_length*(iterations-1):]))
  sentiments_3.append(sentence_sentiment)
  epoch += 1

In [53]:
sentiment_results_3 = []
for sentiment in sentiments_3:
  sentiment_results_3.append(calculateSentiment(sentiment))

converted_sentiments_3 = []
for result in sentiment_results_3:
  converted_sentiments_3.append(result['sent'])

In [60]:
# Replace 'your_file.csv' with the actual file name
output_csv_file = '/content/reddit-processed-output.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)

df['s1'] = converted_sentiments
df['s2'] = converted_sentiments_2
df['s3'] = converted_sentiments_3

# Write the DataFrame with the new column back to a CSV file
df.to_csv(output_csv_file, index=False)